# Data Cleaning and Manipulation

In [ ]:
import pandas as pd
import numpy as np
import datetime
import altair as alt
import plotly.express as px
import geopandas as gpd

# 1. Download Climate Data

Downloaded data from https://www.ncei.noaa.gov/access/monitoring/cei/graph and added the csv files to this project.

Climate variables we are interested in: 

(Definitions are from https://www.ncei.noaa.gov/access/monitoring/cei/definition)

Extremes in Maximum Temperature ... where max temp = daytime temp
Definition: The sum of (a) percentage of the region with maximum temperatures much below normal and
(b) percentage of the region with maximum temperatures much above normal.

Extremes in Minimum Temperature ... where min temp = nighttime temp
Definition: The sum of (a) percentage of the region with minimum temperatures much below normal and
(b) percentage of the region with minimum temperatures much above normal.

Extremes in Days with/without Precip 
Definition: The sum of (a) percentage of the United States with a much greater than normal number of days with precipitation and 
(b) percentage of the United States with a much greater than normal number of days without precipitation.

In [ ]:
#Add climate data
#Northeast data

ne_files = [['ne_maxT_winter.csv','ne_minT_winter.csv','ne_precip_winter.csv'],
            ['ne_maxT_spring.csv','ne_minT_spring.csv','ne_precip_spring.csv' ],
            ['ne_maxT_summer.csv','ne_minT_summer.csv','ne_precip_summer.csv'],
            ['ne_maxT_fall.csv', 'ne_minT_fall.csv', 'ne_precip_fall.csv']]
ne_dfs = []

for file in ne_files:
    season_df = []
    for f in file:
        name=f.split('_')
        df=pd.read_csv(f, skiprows=2, names=[ 'Year', name[1] + ' much above normal', name[1] + ' much below normal'])
        season_df.append(df)
    df['Season'] = name[2].split('.')[0]
    #concat each seasons df together
    season_dfs = pd.concat(objs=season_df, axis=1)
    ne_dfs.append(season_dfs)  

#Concat dfs together
ne = pd.concat(objs=ne_dfs)

#Remove duplicated year columns
ne = ne.loc[:,~ne.columns.duplicated()]

#Add region column
ne["Region"] = 'Northeast'

#ne.head()

In [ ]:
#Southeast data

se_files = [['se_maxT_winter.csv','se_minT_winter.csv','se_precip_winter.csv'],
            ['se_maxT_spring.csv','se_minT_spring.csv','se_precip_spring.csv' ],
            ['se_maxT_summer.csv','se_minT_summer.csv','se_precip_summer.csv'],
            ['se_maxT_fall.csv', 'se_minT_fall.csv', 'se_precip_fall.csv']]
se_dfs = []

for file in se_files:
    season_df = []
    for f in file:
        name=f.split('_')
        df=pd.read_csv(f, skiprows=2, names=[ 'Year', name[1] + ' much above normal', name[1] + ' much below normal'])
        season_df.append(df)
    df['Season'] = name[2].split('.')[0]
    #Concat each season df together
    season_dfs = pd.concat(objs=season_df, axis=1)
    se_dfs.append(season_dfs)
    
#Concat dfs together
se = pd.concat(objs=se_dfs)

#Remove duplicated year columns
se = se.loc[:,~se.columns.duplicated()]

#Add region column
se["Region"] = 'Southeast'

#se.head()

In [ ]:
#Concat northeast and southeast df
climate = pd.concat([se,ne])

#Capitalizing season (to match phen df)
climate['Season'] = climate['Season'].str.capitalize()

#UNCOMMENT TO EXPORT FILE
#climate.to_csv('climate', index = False)

climate

,Year,maxT much above normal,maxT much below normal,minT much above normal,minT much below normal,precip much above normal,precip much below normal,Season,Region
0,1911,1.0,0.0,0.0,1.8,0.0,56.1,Winter,Southeast
1,1912,0.0,38.3,0.0,15.0,11.0,5.1,Winter,Southeast
2,1913,13.0,0.0,20.7,0.0,8.4,10.6,Winter,Southeast
3,1914,0.0,0.9,0.0,0.0,3.0,14.2,Winter,Southeast
4,1915,0.0,43.0,0.0,0.0,21.6,5.2,Winter,Southeast
...,...,...,...,...,...,...,...,...,...
107,2017,71.4,0.0,62.3,0.0,0.0,4.7,Fall,Northeast
108,2018,0.0,17.7,58.0,0.0,95.3,0.0,Fall,Northeast
109,2019,0.0,0.1,1.4,0.0,0.0,0.0,Fall,Northeast
110,2020,25.7,0.0,30.9,0.0,0.0,0.0,Fall,Northeast


# 2. Download Phenophase Data

Downloaded data from https://data.usanpn.org/observations/ and added the csv file to this project.

Phenophases we are interested in: 

(descriptions are from https://www.usanpn.org/files/shared/files/Plant%20and%20Animal%20Phenophase%20Definition%20Supplement.pdf)

First Leaf - (aka breaking leaf buds) "One or more breaking leaf buds are visible on the plant." For lilacs in particular, "In at least 3 locations on the plant, a breaking leaf bud is visible."

First bloom - (aka open flowers) "One or more open, fresh flowers are visible on the plant." For lilacs in particular, "at least half (50%) of the flower clusters have at least one open fresh flower. The lilac flower cluster is a grouping of many, small individual flowers."

Full bloom - (aka full flowering) For lilacs in particular, "For the whole plant, virtually all (95-100%) of the flower clusters no longer have any unopened flowers, but many of the flowers are still fresh and have not withered."

End bloom - (aka end of flowering) For lilacs in particular, "For the whole plant, virtually all (95-100%) of the flowers have withered or dried up and the floral display has ended."

In [ ]:
#read in phenometrics data
phen = pd.read_csv('individual_phenometrics_data.csv')


columns_to_keep = ['Individual_ID', 'Latitude', 'Longitude', 'Elevation_in_Meters', 'State',
    'Common_Name','Phenophase_Description', 'First_Yes_Year','First_Yes_Month',
    'First_Yes_Day', 'First_Yes_DOY', 'Last_Yes_Year', 'Last_Yes_Month', 'Last_Yes_Day', 
    'Last_Yes_DOY', 'Genus']

phen = phen[phen.columns[phen.columns.isin(columns_to_keep)]]

phen

,Latitude,Longitude,Elevation_in_Meters,State,Genus,Common_Name,Individual_ID,Phenophase_Description,First_Yes_Year,First_Yes_Month,First_Yes_Day,First_Yes_DOY,Last_Yes_Year,Last_Yes_Month,Last_Yes_Day,Last_Yes_DOY
0,43.380001,-72.599998,74,VT,Syringa,Red Rothomagensis lilac,4166,First leaf (historic lilac/honeysuckle),1965,5,4,124,1965,5,4,124
1,43.380001,-72.599998,74,VT,Syringa,Red Rothomagensis lilac,4166,First bloom (historic eastern lilac),1965,5,26,146,1965,5,26,146
2,43.380001,-72.599998,74,VT,Syringa,Red Rothomagensis lilac,4166,Full bloom (historic lilac/western honeysuckle),1965,6,8,159,1965,6,8,159
3,44.520000,-73.120003,103,VT,Syringa,Red Rothomagensis lilac,5457,First leaf (historic lilac/honeysuckle),1965,5,1,121,1965,5,1,121
4,44.520000,-73.120003,103,VT,Syringa,Red Rothomagensis lilac,5457,First bloom (historic eastern lilac),1965,5,25,145,1965,5,25,145
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354228,40.628132,-74.039925,5,NY,Acer,Norway maple,300952,Falling leaves,2022,9,20,263,2022,9,20,263
354229,40.628132,-74.039925,5,NY,Acer,Norway maple,300952,Leaves,2022,9,20,263,2022,9,20,263
354230,40.628132,-74.039925,5,NY,Acer,Norway maple,300952,Colored leaves,2022,9,20,263,2022,9,20,263
354231,40.586861,-74.775925,73,NJ,Pinus,eastern white pine,300982,Ripe seed cones,2022,10,2,275,2022,10,2,275


### 2.1 Phenophase Investigation

Phenophase Column Definitions

(definitions are from the "Individual Phenometrics Datafield Descriptions" https://data.usanpn.org/observations/metadata)

Individual_ID = The unique identifier of the individual plant or the animal species at a site for which the series was recorded. Note that for plants, individuals are tracked separately, while for animals, the species as a whole (rather than unique individuals) is tracked at a site. More information can be found in the ancillary data file for "Individual Plant".

Latitude/Longitude = The lat/lon of the site at which the series was recorded. Generally lat/long is calculated from the Google Maps API with a datum of WGS84 (https://developers.google.com/maps), unless a plausible user-defined lat/long was submitted. Information about the datum and source of the lat/long value can be found in the "Site" ancillary data file.

Elevation_in_Meters = The elevation (in meters) of the site at which the series was recorded. Generally elevation is calculated from the Google Maps Elevation API (https://developers.google.com/maps/documentation/elevation/intro), unless a plausible user-defined elevation was submitted. Information about the source of the elevation value can be found in the "Site" ancillary data file. A value of "-9999" indicates the elevation could not be calculated.

State = The U.S. state or territory, Mexican state or Canadian province in which the site is located. The state is calculated from lat/long by the Google Maps Geocoding API (https://developers.google.com/maps/documentation/geocoding/intro). A value of "-9999" indicates the site does not fall within the boundaries of North America.

Genus = The taxonomic genus of the organism for which the series was recorded. Taxonomy follows that in the Integrated Taxonomic Information System (http://itis.gov).

Common_Name = The common name of the species for which the series was recorded. Common names for plants follow those in the USDA PLANTS Database (http://plants.usda.gov), and for animals, in the NatureServe database (http://explorer.natureserve.org).

Phenophase_Description = The descriptive title of the phenophase for which the series was recorded. More information can be found in the ancillary data files for "Phenophase" and "Phenophase Definition".

First_Yes_[Year/Month/Day] = The year/month/day of the month of the first "yes" record of the series.

First_Yes_DOY = The day of year, ranging from 1 to 366, of the first "yes" record of the series.

Last_Yes_[Year/Month/Day] = The year/month/day of the month of the last "yes" record of the series.

Last_Yes_DOY = The day of year, ranging from 1 to 366, of the last "yes" record of the series.

In [ ]:
#checking to see phenophases
phen["Phenophase_Description"].unique()

array(['First leaf (historic lilac/honeysuckle)',
       'First bloom (historic eastern lilac)',
       'Full bloom (historic lilac/western honeysuckle)',
       'End bloom (historic lilac/eastern honeysuckle)',
       'Full leaf (historic lilac/honeysuckle)',
       'First bloom (historic honeysuckle)',
       'Full bloom (historic eastern honeysuckle)',
       'Open flowers (lilac)', 'Full flowering (lilac)',
       'End of flowering (lilac/honeysuckle)',
       'Breaking leaf buds (lilac/honeysuckle)',
       'All leaf buds broken (lilac/honeysuckle)',
       '>=75% of full leaf size (deciduous)', 'Full flowering (50%)',
       'Breaking leaf buds', 'Leaves', 'Open flowers', 'Ripe fruits',
       'Open flowers (1 location)', '>=50% of leaves colored (deciduous)',
       'All leaves colored (deciduous)',
       '>=50% of leaves fallen (deciduous)',
       'All leaves fallen (deciduous)',
       'Full pollen release (angiosperms)', 'Pollen release (flowers)',
       'Young needles (pi

In [ ]:
#need to drop full flowering (50%) as we are interested in 95-100% for full flowering
phen.drop(phen[phen['Phenophase_Description'] == "Full flowering (50%)"].index, inplace = True)

In [ ]:
#Narrowing down phenophases of interest
#Update the phenophase names to help merge the historical with the more recent observations

#remove portion of name in parentheses
phen['Phenophase_Description'] = phen['Phenophase_Description'].apply(lambda x: x.split(' (')[0])

#replace more recent phenophase terms to be consistent with historical terms
phen['Phenophase_Description'] = phen['Phenophase_Description'].replace(
            ['End of flowering','Full flowering','Open flowers', 'Breaking leaf buds'],
            ['End bloom','Full bloom', 'First bloom', 'First leaf'])

In [ ]:
#filter to just get the phenophases we are interested in 
temp = ['First leaf', 'End of flowering', 'End bloom', 'First bloom', 'Full bloom']
phen = phen[phen['Phenophase_Description'].isin(temp)]

### 2.2 Creating Date and Region Columns

In [ ]:
#Renaming first columns to use for datetime
phen = phen.rename({'First_Yes_Year': 'year', 'First_Yes_Month': 'month', 'First_Yes_Day':'day'}, axis=1)

#Converting to first date
phen['First date'] = pd.to_datetime(phen[['year', 'month', 'day']])

#Getting rid of the time component in date 
phen['First date'] = phen['First date'].dt.date

#Dropping extra data columns
phen = phen.drop(labels=['year', 'month', 'day'], axis=1)

#renaming last columsn to use for datetime
phen = phen.rename({'Last_Yes_Year': 'Year', 'Last_Yes_Month': 'month', 'Last_Yes_Day':'day'}, axis=1)

#Converting to last date
phen['Last date'] = pd.to_datetime(phen[['Year', 'month', 'day']])

#Getting rid of the time component in date 
phen['Last date'] = phen['Last date'].dt.date

#Dropping extra columns
phen = phen.drop(labels=['month', 'day', 'Last_Yes_DOY', 'Last date'], axis=1)

phen['First date'] = phen['First date'].apply(lambda x: '{:%m-%d}'.format(datetime.datetime.strptime(str(x), '%Y-%m-%d')))
#phen

In [ ]:
#Defining Regions
northeast = ['ME', 'VT', 'NH', 'MA', 'RI', 'CT', 'NJ', 'DE', 'MD', 'PA', 'NY']
southeast = ['VA, NC, SC, GA, FL, AL']

phen['Region'] = np.where(phen['State'].isin(northeast), 'Northeast','Southeast')

In [ ]:
#IMPORTANT NOTE: We have a map that shows both Syringa and Lonicera for NE and SE 
#and the map is created right before the next steps

#UNCOMMENT TO EXPORT FILE
#phen.to_csv('phen_map_df', index = False)

### 2.3 Slicing and Pivoting Data

In [ ]:
#subset to just lilacs
phen = phen[phen['Genus'] == 'Syringa']

#only using Northeast
phen = phen[phen['Region'] == 'Northeast']

#pivoting into wide format
#each row will becaome a single plant's info for that year

columns = list(phen.columns)
remove = ['Phenophase_Description', 'First date', 'First_Yes_DOY']
columns = [i for i in columns if i not in remove]


phen = phen.pivot_table(values = ['First date', 'First_Yes_DOY'], index = columns, columns = 'Phenophase_Description', aggfunc = min).reset_index()
#phen

In [ ]:
#removing the multilevel indexing and renaming columns
phen['First leaf date'] = phen.loc[:, ('First date',['First leaf'])]
phen['First bloom date'] = phen.loc[:, ('First date',['First bloom'])]
phen['Full bloom date'] = phen.loc[:, ('First date',['Full bloom'])]
phen['End bloom date'] = phen.loc[:, ('First date',['End bloom'])]

phen['First leaf DOY'] = phen.loc[:, ('First_Yes_DOY',['First leaf'])]
phen['First bloom DOY'] = phen.loc[:, ('First_Yes_DOY',['First bloom'])]
phen['Full bloom DOY'] = phen.loc[:, ('First_Yes_DOY',['Full bloom'])]
phen['End bloom DOY'] = phen.loc[:, ('First_Yes_DOY',['End bloom'])]

phen = phen.drop(columns = ['First date', 'First_Yes_DOY'])
phen.columns = phen.columns.droplevel(1)

/shared-libs/python3.10/py/lib/python3.10/site-packages/pandas/core/generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


### 2.4 Adding Season Column

In [ ]:
#Defining seasons for the phenophases

season_dict = { '01': 'Winter',
                '02': 'Winter',
                '03': 'Spring', 
                '04': 'Spring',
                '05': 'Spring',
                '06': 'Summer',
                '07': 'Summer',
                '08': 'Summer',
                '09': 'Fall',
                '10': 'Fall',
                '11': 'Fall',
                '12': 'Winter'}

In [ ]:
#phen before adding season column for join
#UNCOMMENT TO EXPORT FILE
#phen.to_csv('phen', index = False)

#we are defining season as season the "first bloom" phenophase occurred
#create df that joins climate data on season of first bloom for each plant
phen['Season'] = phen['First bloom date'].apply(lambda x: 'Spring' if pd.isna(x) else season_dict[str(x).split('-')[0]])

In [ ]:
#how many indiv lilac plants across how many states in the NE?
phen.groupby("State")["Individual_ID"].count()

State
CT     272
DE      40
MA     362
MD      83
ME     398
NH     167
NJ     298
NY    1213
PA     407
RI      31
VT     876
Name: Individual_ID, dtype: int64

# 3. Combining Data Sets

In [ ]:
updated_df = climate.merge(right=phen, how='inner', on=['Year', 'Region', 'Season'])

#UNCOMMENT TO EXPORT FILE
updated_df.to_csv('updated_df', index = False)

updated_df

,Year,maxT much above normal,maxT much below normal,minT much above normal,minT much below normal,precip much above normal,precip much below normal,Season,Region,Latitude,...,Common_Name,Individual_ID,First leaf date,First bloom date,Full bloom date,End bloom date,First leaf DOY,First bloom DOY,Full bloom DOY,End bloom DOY
0,1965,0.0,0.1,0.0,0.0,0.0,20.6,Spring,Northeast,41.470001,...,Red Rothomagensis lilac,56075,04-28,05-17,05-20,NaN,118.0,137.0,140.0,NaN
1,1965,0.0,0.1,0.0,0.0,0.0,20.6,Spring,Northeast,41.799999,...,Red Rothomagensis lilac,56077,04-24,05-16,05-25,NaN,114.0,136.0,145.0,NaN
2,1965,0.0,0.1,0.0,0.0,0.0,20.6,Spring,Northeast,41.799999,...,Red Rothomagensis lilac,56081,04-17,05-10,05-17,NaN,107.0,130.0,137.0,NaN
3,1965,0.0,0.1,0.0,0.0,0.0,20.6,Spring,Northeast,42.830002,...,Red Rothomagensis lilac,56266,04-26,05-20,05-27,NaN,116.0,140.0,147.0,NaN
4,1965,0.0,0.1,0.0,0.0,0.0,20.6,Spring,Northeast,42.880001,...,Red Rothomagensis lilac,56195,04-29,05-20,05-25,NaN,119.0,140.0,145.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4142,2020,79.2,0.0,83.9,0.0,0.0,10.3,Summer,Northeast,45.195858,...,common lilac,12469,05-06,06-03,06-10,06-24,127.0,155.0,162.0,176.0
4143,2021,12.9,0.0,97.9,0.0,61.1,0.0,Summer,Northeast,43.694210,...,common lilac,271837,NaN,06-01,06-01,06-07,NaN,152.0,152.0,158.0
4144,2020,25.7,0.0,30.9,0.0,0.0,0.0,Fall,Northeast,42.921581,...,common lilac,249679,NaN,09-25,09-25,11-13,NaN,269.0,269.0,318.0
4145,2021,38.1,0.0,99.8,0.0,38.9,0.0,Fall,Northeast,39.935772,...,common lilac,282376,NaN,11-06,NaN,NaN,NaN,310.0,NaN,NaN


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=81a2ddfb-5d15-4e38-b0f3-1ce67090c5b7' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>